In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import math
from random import shuffle
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer


t5base = SentenceTransformer('sentence-transformers/sentence-t5-base')
t5large = SentenceTransformer('sentence-transformers/sentence-t5-large')
t5xl = SentenceTransformer('sentence-transformers/sentence-t5-xl')
t5xxl = SentenceTransformer('sentence-transformers/sentence-t5-xxl')

In [2]:
def encode_t5base(sent): return t5base.encode([sent])[0]
def encode_t5large(sent): return t5large.encode([sent])[0]
def encode_t5xl(sent): return t5xl.encode([sent])[0]
def encode_t5xxl(sent): return t5xxl.encode([sent])[0]

def cosine_similarity(x, y): 
    if len(x) != len(y): return None
    dot_product = np.dot(x, y)
    magnitude_x = np.sqrt(np.sum(x**2)) 
    magnitude_y = np.sqrt(np.sum(y**2))
    return dot_product / (magnitude_x * magnitude_y)

social = ["totalitarian", "authoritarian", "statist", "liberal", "libertarian", "anarchist"]
economic = ["communist", "socialist", "progressive", "regulated capitalist", "capitalist", "laissez-faire"]
compare_to = "A free economy is effective with strong guidance"

belief_feats = {
    'base': [],
    'large': [],
    'xl': [],
    'xxl': []
}

for a in social:
    base_arr, large_arr, xl_arr, xxl_arr = [], [], [], []
    for h in economic:
        belief = "The best economy is {0} and the best government is {1}".format(h,a)
        base_arr.append(encode_t5base(belief))
        large_arr.append(encode_t5large(belief))
        xl_arr.append(encode_t5xl(belief))
        xxl_arr.append(encode_t5xxl(belief))

    belief_feats['base'].append(base_arr)
    belief_feats['large'].append(large_arr)
    belief_feats['xl'].append(xl_arr)
    belief_feats['xxl'].append(xxl_arr)    

In [3]:
def hot_take_plot(statement, idx):
    def plot(model, embed_fn):
        feats = embed_fn(statement)
        max_sim = -2
        min_sim = 2
        arrs = []
        for x in belief_feats[model]:
            arr = []
            for y in x:
                score = cosine_similarity(feats, y)
                max_sim = max(max_sim, score)
                min_sim = min(min_sim, score)
                arr.append(score)
            arrs.append(arr)
        
        sns.set(font_scale=1.2)
        g = sns.heatmap(
            arrs,
            xticklabels=economic,
            yticklabels=social,
            vmin=min_sim,
            vmax=max_sim,
            cmap="YlOrRd")
        g.set_title("T5-{0}: \"{1}\"".format(model, statement))
        plt.savefig("./static/exports/t5-{0}-{1}.png".format(model, idx), bbox_inches = 'tight')
        plt.clf()
    
    plot('base', encode_t5base)
    plot('large', encode_t5large)
    plot('xl', encode_t5xl)
    plot('xxl', encode_t5xxl)
    
    outfile = open("./static/exports/runs.txt", "a")
    statement = statement.replace("\n"," ")
    outfile.write(str(idx)+": "+statement+"\n")
    outfile.close()

hot_take_plot("The general public cannot be trusted to use AI responsibly", 0)

<Figure size 640x480 with 0 Axes>

In [ ]:
from flask import Flask, request
from flask_cors import CORS
import simplejson as json

with open("./static/exports/runs.txt") as f:
    req_idx = (len([x for x in f]))
    print("Reqs: ", req_idx)

app = Flask(__name__, static_url_path='/public')
CORS(app)
@app.route('/')
def root():
    return app.send_static_file('index.html')

@app.route('/<path:path>')
def static_file(path):
    return app.send_static_file(path)

@app.route("/calcHotTake")
def gd():
    global req_idx
    req_idx += 1
    
    args = request.args
    if 'statement' not in args:
        return json.dumps({'status':400, 'message':'No statement'})
    
    statement = args['statement']
    hot_take_plot(statement, req_idx)
    return json.dumps({'status':200, 'req_idx': req_idx})

@app.errorhandler(404)
def not_found_error(error):
    return "error 404"

if __name__ == '__main__':
    port = 9102
    app.run(host='0.0.0.0', port=port)

Reqs:  1
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:9102
 * Running on http://192.168.0.135:9102
Press CTRL+C to quit
127.0.0.1 - - [22/Jul/2023 08:31:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:31:58] "GET /images/loading.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:31:58] "GET /styles.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:31:58] "GET /images/base.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:31:58] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:31:58] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:31:58] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:32:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:32:21] "GET /styles.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:32:21] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:32:21] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:32:21] "GET /images/loading.gif HTTP/1.1" 200 -
127

127.0.0.1 - - [22/Jul/2023 08:49:01] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:49:01] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:49:01] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:49:01] "GET /exports/runs.txt HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:49:39] "GET /?i=0 HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:49:39] "GET /styles.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:49:39] "GET /images/loading.gif HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:49:39] "GET /images/base.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:49:39] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:49:39] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:49:39] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:49:39] "GET /exports/runs.txt HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:51:12] "GET /?i=0 HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:51:12] "GET /styles.

127.0.0.1 - - [22/Jul/2023 08:54:43] "GET /images/base.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:43] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:43] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:43] "GET /exports/runs.txt HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:43] "GET /exports/t5-base-0.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:54:43] "GET /exports/t5-large-0.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:54:43] "GET /exports/t5-xxl-0.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:43] "GET /exports/t5-xl-0.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:54:46] "GET /styles.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:46] "GET /images/loading.gif HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:46] "GET /images/base.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 08:54:46] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1 - - [22/J

127.0.0.1 - - [22/Jul/2023 09:03:43] "GET /images/loading.gif HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 09:03:44] "GET /calcHotTake?statement=I%20need%20help HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:03:44] "GET /exports/t5-base-6.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:03:44] "GET /exports/t5-large-6.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:03:44] "GET /exports/t5-xl-6.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:03:44] "GET /exports/t5-xxl-6.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:04:01] "GET /calcHotTake?statement=I%20am%20helpful HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:04:02] "GET /exports/t5-base-7.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:04:02] "GET /exports/t5-xl-7.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:04:02] "GET /exports/t5-large-7.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:04:02] "GET /exports/t5-xxl-7.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 09:04:17] "GET /calcHotTake?statement=It%20takes%20a%20v

...

127.0.0.1 - - [22/Jul/2023 12:19:15] "GET /exports/t5-xxl-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /?i=12 HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /styles.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /images/base.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /images/loading.gif HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /images/git.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /images/colab.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /exports/runs.txt HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /exports/t5-base-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:19:32] "GET /exports/t5-large-12.png HTTP/1.1" 304 -
127.0.0.1 - - [2

127.0.0.1 - - [22/Jul/2023 12:34:55] "GET /exports/runs.txt HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:34:55] "GET /exports/t5-base-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:34:55] "GET /exports/t5-xl-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:34:55] "GET /exports/t5-large-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:34:55] "GET /exports/t5-xxl-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:35:14] "GET /?i=12 HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 12:35:14] "GET /styles.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 12:35:14] "GET /images/loading.gif HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 12:35:14] "GET /images/base.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 12:35:14] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:35:14] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:35:14] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:35:14] "GET /images/git.png HTTP/1.1" 304 -
127.0.0.1 - 

127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /images/base.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /images/git.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /exports/runs.txt HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /images/colab.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /exports/t5-base-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /exports/t5-large-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /exports/t5-xl-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:45:36] "GET /exports/t5-xxl-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:46:09] "GET /?i=12 HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:46:09] "GET /styles.css HTTP/1.1" 200 -
127.0.0.1 - - 

127.0.0.1 - - [22/Jul/2023 12:53:11] "GET /exports/t5-xl-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:11] "GET /exports/t5-xxl-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:11] "GET /exports/t5-large-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /?i=12 HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /styles.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /images/loading.gif HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /images/base.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /images/xl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /images/git.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /images/colab.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 12:53:30] "GET /exports/runs.txt HTTP/1.1" 200 -
127.0.0.1 - - [22/

127.0.0.1 - - [22/Jul/2023 13:00:28] "GET /images/xxl.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 13:00:28] "GET /images/git.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 13:00:28] "GET /images/colab.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 13:00:28] "GET /exports/runs.txt HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 13:00:28] "GET /exports/t5-base-12.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 13:00:28] "GET /exports/t5-large-12.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 13:00:28] "GET /exports/t5-xl-12.png HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 13:00:28] "GET /exports/t5-xxl-12.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 13:01:15] "GET /?i=12 HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 13:01:15] "GET /styles.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 13:01:15] "GET /images/base.jpeg HTTP/1.1" 304 -
127.0.0.1 - - [22/Jul/2023 13:01:15] "GET /images/loading.gif HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 13:01:15] "GET /images/large.png HTTP/1.1" 304 -
127.0.0.1